In [6]:
from langchain_groq import ChatGroq

In [8]:
llm = ChatGroq(
    model_name="llama-3.1-70b-versatile",
    groq_api_key='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',
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

response = llm.invoke("The first person to land on moon was...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped onto the lunar surface on July 20, 1969, as part of the Apollo 11 mission.


In [11]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-43218?from=job%20search%20funnel")
page_data = loader.load().pop().page_content
page_data

"Apply for Lead Analytics Engineer\n\nSearch JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsData & AnalyticsLead Analytics EngineerShanghai, China MainlandBecome a Part of the NIKE, Inc. Team\

In [14]:
from langchain_core.prompts import PromptTemplate
prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WESITE:
    {page_data}
    ### INSTRUCTION
    The scraped text is from the career's page of a website.
    Your job is to extract job postings and return them in JSON format containing 
    following keys: 'role', 'experience', 'skills' and 'description'.
    Only return the valid JSON.
    ### VALID JSON (NO PREAMBLE):
    """
)

chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data': page_data})
print(res.content.strip())

```json
{
  "role": "Lead Analytics Engineer",
  "experience": "Minimum 8 years of experience in data management, encompassing data capture, data governance, and master data management",
  "skills": [
    "Bachelor's degree in Computer Science, Information Systems, or a related field",
    "Strong understanding of data management principles and best practices",
    "Proficient with data capture tools and technologies",
    "Experience with Sensors is a plus",
    "Excellent command over SQL and other data query languages",
    "Outstanding problem-solving and analytical skills",
    "Extremely strong and mature communication skills and fluent in English"
  ],
  "description": "The data governance management position is a critical role of GC Data&AI team that needs to cover several different data aspects. We are seeking a highly motivated and experienced Data Management Professional to join our dynamic team. This role is crucial in ensuring the efficient management and utilization of da

In [19]:
from langchain_core.output_parsers import JsonOutputParser
json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Lead Analytics Engineer',
 'experience': 'Minimum 8 years of experience in data management, encompassing data capture, data governance, and master data management',
 'skills': ["Bachelor's degree in Computer Science, Information Systems, or a related field",
  'Strong understanding of data management principles and best practices',
  'Proficient with data capture tools and technologies',
  'Experience with Sensors is a plus',
  'Excellent command over SQL and other data query languages',
  'Outstanding problem-solving and analytical skills',
  'Extremely strong and mature communication skills and fluent in English'],
 'description': 'The data governance management position is a critical role of GC Data&AI team that needs to cover several different data aspects. We are seeking a highly motivated and experienced Data Management Professional to join our dynamic team. This role is crucial in ensuring the efficient management and utilization of data within our organization.'}

In [21]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [39]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for index, row in df.iterrows():
        collection.add(documents=row['Techstack'],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])
        link_lists = row["Links"]

In [36]:
links = collection.query(
    query_texts = [
        "Experience in Python",
        "Experience in React"
    ], n_results = 2
)


[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/react-native-portfolio'}]]

In [34]:
job = json_res
job['skills']

["Bachelor's degree in Computer Science, Information Systems, or a related field",
 'Strong understanding of data management principles and best practices',
 'Proficient with data capture tools and technologies',
 'Experience with Sensors is a plus',
 'Excellent command over SQL and other data query languages',
 'Outstanding problem-solving and analytical skills',
 'Extremely strong and mature communication skills and fluent in English']

In [40]:
 prompt_email = PromptTemplate.from_template(
            """
            ### JOB DESCRIPTION:
            {job_description}

            ### INSTRUCTION:
            I am a Software Development student at Sheridan College, currently in my third year. I have developed many projects and have experience from two co-op terms. Over my experience, I have empowered numerous enterprises with tailored solutions, fostering scalability, process optimization, cost reduction, and heightened overall efficiency.

            Could you please help me write a cold email to a manager regarding a job opportunity? 
            The email should describe my capabilities in fulfilling their needs and include the most relevant projects from the following links to showcase my portfolio: {link_list}. Remember, my name is Jeenitesh.
            Do not provide a preamble.
            ### EMAIL (NO PREAMBLE):

            """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Application for Lead Analytics Engineer Position

Dear Hiring Manager,

I am writing to express my interest in the Lead Analytics Engineer position at your esteemed organization. As a third-year Software Development student at Sheridan College, I am confident that my skills and experience make me an ideal candidate for this role.

With a strong foundation in computer science and information systems, I have developed a solid understanding of data management principles and best practices. My experience in data capture, data governance, and master data management has been demonstrated through various projects, including those that utilize machine learning, Python, and TensorFlow. I am excited about the opportunity to bring my skills and expertise to your team and contribute to the efficient management and utilization of data within your organization.

Some of my notable projects include:

* A machine learning project that leverages Python and TensorFlow to develop predictive mode